<a href="https://colab.research.google.com/github/jagdaleyash/phishingdetectiontool/blob/main/Phishing_Email_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

data = pd.read_csv('Phishing_paper1.csv')

print(data.head())


   Total Number of Characters C  Vocabulary richness W/C  Account  Access  \
0                          1673                 0.128512        2       0   
1                          4465                 0.137738        0       0   
2                          6813                 0.095993        0       0   
3                          1518                 0.109354        0       0   
4                          1881                 0.126528        7       3   

   Bank  Credit  Click  Identity  Inconvenience  Information  ...  Password  \
0     0       0      1         0              0            1  ...         1   
1     0       0      0         0              0            4  ...         0   
2     0       2      0         0              0            2  ...         0   
3     0       1      1         0              0            0  ...         0   
4     0       0      0         1              0            2  ...         0   

   Recently  Risk  Social  Security  Service  Suspended  \
0  

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('punkt')

nltk.download('stopwords')
from nltk.tokenize import word_tokenize

# load the data
data = pd.read_csv('Phishing_paper1.csv', header=None, names=['url', 'label'])

# drop the first row as it contains column names
data = data.drop(data.index[0])

# remove rows with missing values
data = data.dropna()

# convert label column to numeric values
data['label'] = pd.to_numeric(data['label'])

# replace NaN values with empty strings
data['url'].fillna('', inplace=True)

# convert url column to string
data['url'] = data['url'].astype(str)

# define function to clean url
def clean_url(url):
    url = url.lower()
    url = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))','URL',url)
    url = re.sub(r'\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b', 'IPADDRESS', url)
    url = re.sub(r'[^\w\s]','',url)
    url = re.sub(r'\s+', ' ', url)
    return url

data['url'] = data['url'].apply(clean_url)

# tokenize the urls
def tokenize_url(url):
    tokens = word_tokenize(url)
    return tokens

data['tokens'] = data['url'].apply(tokenize_url)

# remove stop words
stop_words = set(stopwords.words('english'))

def remove_stop_words(tokens):
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return filtered_tokens

data['tokens'] = data['tokens'].apply(remove_stop_words)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-13-3228d249da6e>:11: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Phishing_paper1.csv', header=None, names=['url', 'label'])


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# create CountVectorizer object
vectorizer = CountVectorizer()

# join the tokens into a single string
data['tokens'] = data['tokens'].apply(lambda x: ' '.join(x))

# fit and transform the tokenized URLs
X = vectorizer.fit_transform(data['tokens'])

# print the shape of the vectorized URLs
print('Vectorized URLs shape:', X.shape)


Vectorized URLs shape: (525754, 2121)


In [ ]:
from sklearn.model_selection import train_test_split

# create training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, data['label'], test_size=0.3, random_state=42)

# print the shape of the training and testing sets
print('Training set shape:', X_train.shape, y_train.shape)
print('Testing set shape:', X_test.shape, y_test.shape)


Training set shape: (368027, 2121) (368027,)
Testing set shape: (157727, 2121) (157727,)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# create logistic regression model with a high maximum number of iterations
logreg = LogisticRegression(max_iter=10000)

# fit the model to the training data
logreg.fit(X_train, y_train)

# predict on the testing data
y_pred = logreg.predict(X_test)

# calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)

# print accuracy score
print('Accuracy:', accuracy)

Accuracy: 0.9843019901475334


In [ ]:
# create logistic regression model with a high maximum number of iterations
logreg = LogisticRegression(max_iter=10000)

# fit the model to the entire preprocessed dataset
logreg.fit(X_train, y_train)

# preprocess the email
email = 'http://example.com'

email = clean_url(email)
tokens = tokenize_url(email)
tokens = remove_stop_words(tokens)
email = ' '.join(tokens)

# vectorize the preprocessed email
email_vector = vectorizer.transform([email])

# predict whether the email is phishing or not using the trained model
prediction = logreg.predict(email_vector)

if prediction == 1:
    print('This email is a phishing email')
else:
    print('This email is not a phishing email')


This email is not a phishing email


In [ ]:
from sklearn.metrics import classification_report

# compute classification report
report = classification_report(y_test, y_pred)
print(report)


              precision    recall  f1-score   support

           0       0.98      1.00      0.99    155251
           1       0.00      0.00      0.00      2476

    accuracy                           0.98    157727
   macro avg       0.49      0.50      0.50    157727
weighted avg       0.97      0.98      0.98    157727



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# example email
email = "Dear customer, we have detected suspicious activity on your account. Please click on the link to verify your account: http://example.com/verify?id=1kjjk23"

# preprocess the email
email = clean_url(email)
tokens = tokenize_url(email)
tokens = remove_stop_words(tokens)
email = ' '.join(tokens)

# vectorize the email using the trained vectorizer
email_vector = vectorizer.transform([email])

# predict whether the email is phishing or not using the trained model
prediction = logreg.predict(email_vector)

if prediction == 1:
    print("The email is a phishing email.")
else:
    print("The email is not a phishing email.")


The email is not a phishing email.
